<h1> Capstone Project - The Battle of Neighborhoods (Week 1 and Week 2) </h1>
<p>Now that you have been equipped with the skills and the tools to use location data to explore a geographical location, over the course of two weeks, you will have the opportunity to be as creative as you want and come up with an idea to leverage the Foursquare location data to explore or compare neighborhoods or cities of your choice or to come up with a problem that you can use the Foursquare location data to solve. If you cannot think of an idea or a problem, here are some ideas to get you started:</p>

<li> In Module 3, we explored New York City and the city of Toronto and segmented and clustered their neighborhoods. Both cities are very diverse and are the financial capitals of their respective countries. One interesting idea would be to compare the neighborhoods of the two cities and determine how similar or dissimilar they are. Is New York City more like Toronto or Paris or some other multicultural city? I will leave it to you to refine this idea.</li>

<li>In a city of your choice, if someone is looking to open a restaurant, where would you recommend that they open it? Similarly, if a contractor is trying to start their own business, where would you recommend that they setup their office?</li>
<p> hese are just a couple of many ideas and problems that can be solved using location data in addition to other datasets. No matter what you decide to do, make sure to provide sufficient justification of why you think what you want to do or solve is important and why would a client or a group of people be interested in your project. </p>


<h1>The Battle of Neighborhoods</h1>

<h3>IBM Data Science Professional Certificate Course</h3>
<h3>Capstone Project</h3>
<h5>Zaw Toe Myint</h5>
<h5>January 2021</h5>

<h2>Introduction</h2>
<p>Yangon is the capital city of Myanmar with over seven million population. It is located at the strategic place and central to all kind of major business such as banking, trading, hub for travel and tourism, entertainment, food and beverage, etc. Competition is very demanding for all business and data-driven decision making and planning is the only way for the benefits for the new business. This is the reason why the data scientists are getting more popular to generate valuable inputs for new business and providing consultation services for market analysis and business evaluation.</p>


<h2>Problem Statement</h2>
<p>One of my clients would like to invest his new business in food and beverage sectors and would like to open new restaurant in Yangon city. But he neither know how many of restaurants has been operating in Yangon nor where exactly and which type of food should be selected for his new restaurant. In this regard, I was asked to assist with available information for new business planning. According to this request, I got started make ideation to be more understanding on the business and explored the data requirement for appropriate report for this assignment.</p>

<h2>Data Sources</h2>
<p>First of all, I do need the list of restaurants, types of food, locations, and the distribution of different restaurants across the Yangon city. So, decided to use maps API supported by fourSquare.com and python libraries such as pandas, numpy for data frame and folium library for generation of map. Ski-learn was the additional library for clustering. Python geocoder library was also essential for getting geographical coordinates of neighborhoods.
Prior to the data wrangling, I downloaded townships and geocoordinates data from http://themimu.info/place-codes in which townships data, postal code, latitude and longitude data was provided. I removed all the data apart from Yangon data. </p>

<h2> 1. Importing Required Libraries<h2>

In [179]:
import numpy as np 
import pandas as pd 
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.width', None)
import matplotlib.pyplot
import seaborn as sns
import json 
!conda install -c conda-forge geopy --yes
from geopy.geocoders import Nominatim 
import requests 
from pandas.io.json import json_normalize 

import matplotlib.cm as cm
import matplotlib.colors as colors
from sklearn.cluster import KMeans
import folium 

print('Libraries imported.')

Solving environment: done

# All requested packages already installed.

Libraries imported.


<h2> 2. Import Towhship Data including Geocoordinates</h2>
<p>In the beginning, I first downloaded location data from the http://themimu.info/place-codes and generated the data-frame. It was observed that total number of thirty (30) districts within Yangon Division with respective post codes and geocoordinates. The below figure shows the first five line of the list of districts.</p>
<p>Secondly, the map was generated by using python geocoder library and visualized the location on the map with folium library. </p>

In [180]:
url = "/Users/zawtoemyint/OneDrive/Learning and Reading/IT_Data Science_IBM/9. Applied Data Science Capstone/postal_code_ygn.xlsx"
df = pd.read_excel(url)
df.head()

,Region,Township,District,Postcode,latitude,longitude
0,Yangon Division,Ahlone,Ahlone,11121,16.782398,96.127863
1,Yangon Division,Bahan,Bahan,11201,16.815427,96.156112
2,Yangon Division,Botahtaung,Botahtaung,11162,16.771967,96.169709
3,Yangon Division,Dagon,Dagon,11191,16.794952,96.146900
4,Yangon Division,Dagon Myothit (North),Dagon Myothit (North),11421,16.877740,96.190790


In [181]:
print('Shape : ', df.shape)

Shape :  (30, 6)


In [182]:
address = 'Yangon'
geolocator = Nominatim(user_agent="Yangon_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Yangon are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Yangon are 16.7967129, 96.1609916.


In [183]:
map_Yangon = folium.Map(location=[latitude, longitude], zoom_start=10)
map_Yangon

In [184]:
for lat, lng, label in zip(
    df['latitude'], 
    df['longitude'], 
    df['District']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=10,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_Yangon)
map_Yangon

<p> After linking up with FourSquare data with respective neighborhoods, total of 272 venues were observed within 30 districts. Among those, there were 85 unique categories identified after making group within respective districts.<p>

In [185]:
CLIENT_ID = 'Y5UW5VZNDWUMB2RJWNSWZ2TJIWBOEO1M2DG032ISKZNTH44R'
CLIENT_SECRET = 'EXKPSOSDKETT0Y5H1SZQGSDHVAXQ0JN1GWO0O2TWAB2P0GCG' 
VERSION = '20180605'
LIMIT = 100 
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)
df.loc[0, 'District']
radius = 500
url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, radius, LIMIT)
url

Your credentails:
CLIENT_ID: Y5UW5VZNDWUMB2RJWNSWZ2TJIWBOEO1M2DG032ISKZNTH44R
CLIENT_SECRET:EXKPSOSDKETT0Y5H1SZQGSDHVAXQ0JN1GWO0O2TWAB2P0GCG


'https://api.foursquare.com/v2/venues/search?client_id=Y5UW5VZNDWUMB2RJWNSWZ2TJIWBOEO1M2DG032ISKZNTH44R&client_secret=EXKPSOSDKETT0Y5H1SZQGSDHVAXQ0JN1GWO0O2TWAB2P0GCG&ll=16.7967129,96.1609916&v=20180605&radius=500&limit=100'

In [186]:
def getNearbyVenues(names, latitudes,longitudes, radius=500, limit =100):
    venues_list=[]
    for name, lat, lng in zip(names,latitudes,longitudes):
        print(name)
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            limit)
        results = requests.get(url).json()["response"]['groups'][0]['items']
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])
    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['District', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)
print("It's okay")

It's okay


In [189]:
df = getNearbyVenues(names=df['District'],
                                   latitudes=df['latitude'],
                                   longitudes=df['longitude'],
                                  )

Ahlone
Bahan
Botahtaung
Dagon
Dagon Myothit (North)
Dagon Myothit (South)
Dawbon
Hlaing
Hlaing Thar Yar
Insein
Mayangone
Kamayut
Kyauk Tan
Kyauktada
Kyimyindaing
Lanmadaw
Mingalar Taung Nyunt
Mingalardon
Okkalapa (North)
Okkalapa (South)
Pabedan
Pazundaung
Sanchaung
Shwe Pyi Thar
Tamwe
Tanyin
Tharkayta
Thingangyun
Yangon
Yankin


In [190]:
print(df.shape)
df.head()

(272, 7)


,District,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Ahlone,16.782398,96.127863,Sein Gay Har @ Pyay Road,16.781087,96.127681,Shopping Mall
1,Ahlone,16.782398,96.127863,Urban 86 Food Court,16.782257,96.126869,Food Stand
2,Ahlone,16.782398,96.127863,Meringue Café,16.786160,96.128021,Café
3,Ahlone,16.782398,96.127863,Ahlone Drink,16.786057,96.129984,Juice Bar
4,Ahlone,16.782398,96.127863,ေရြွနန္းေတာ္ မုန့္တီ,16.784361,96.132085,Noodle House


In [191]:
df.groupby('District').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
District,,,,,,
Ahlone,5,5,5,5,5,5
Bahan,25,25,25,25,25,25
Botahtaung,16,16,16,16,16,16
Dagon,15,15,15,15,15,15
Dagon Myothit (South),1,1,1,1,1,1
Dawbon,1,1,1,1,1,1
Hlaing,4,4,4,4,4,4
Insein,1,1,1,1,1,1
Kamayut,10,10,10,10,10,10


In [192]:
print('There are {} uniques categories.'.format(len(df['Venue Category'].unique())))

There are 85 uniques categories.


In [193]:
yangon_onehot = pd.get_dummies(df[['Venue Category']], prefix="", prefix_sep="")

yangon_onehot['Neighborhood'] = df['District'] 

fixed_columns = [yangon_onehot.columns[-1]] + list(yangon_onehot.columns[:-1])
yangon_onehot = yangon_onehot[fixed_columns]

yangon_onehot.head()


,Whisky Bar,Accessories Store,American Restaurant,Asian Restaurant,Athletics & Sports,Australian Restaurant,Auto Workshop,BBQ Joint,Bakery,Bar,Bed & Breakfast,Bistro,Boxing Gym,Breakfast Spot,Bubble Tea Shop,Buffet,Burger Joint,Burmese Restaurant,Burrito Place,Bus Station,Bus Stop,Café,Chinese Restaurant,Clothing Store,Cocktail Bar,Coffee Shop,Convenience Store,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Donut Shop,Dumpling Restaurant,Electronics Store,Fast Food Restaurant,Fish Market,Food Court,Food Stand,Food Truck,Fried Chicken Joint,Garden,Gift Shop,Gym,Gym / Fitness Center,Hostel,Hotel,Hotel Bar,IT Services,Ice Cream Shop,Indian Restaurant,Indonesian Restaurant,Intersection,Japanese Restaurant,Juice Bar,Korean Restaurant,Lake,Lounge,Market,Massage Studio,Movie Theater,Multiplex,Neighborhood,Noodle House,Park,Photography Studio,Pizza Place,Pub,Restaurant,Sandwich Place,Seafood Restaurant,Shopping Mall,Shopping Plaza,Snack Place,Soup Place,Spa,Steakhouse,Supermarket,Sushi Restaurant,Szechuan Restaurant,Tea Room,Thai Restaurant,Vietnamese Restaurant,Warehouse Store
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,Ahlone,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,Ahlone,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,Ahlone,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,Ahlone,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,Ahlone,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [195]:
yangon_grouped = yangon_onehot.groupby('Neighborhood').mean().reset_index()

yangon_grouped

,Neighborhood,Whisky Bar,Accessories Store,American Restaurant,Asian Restaurant,Athletics & Sports,Australian Restaurant,Auto Workshop,BBQ Joint,Bakery,Bar,Bed & Breakfast,Bistro,Boxing Gym,Breakfast Spot,Bubble Tea Shop,Buffet,Burger Joint,Burmese Restaurant,Burrito Place,Bus Station,Bus Stop,Café,Chinese Restaurant,Clothing Store,Cocktail Bar,Coffee Shop,Convenience Store,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Donut Shop,Dumpling Restaurant,Electronics Store,Fast Food Restaurant,Fish Market,Food Court,Food Stand,Food Truck,Fried Chicken Joint,Garden,Gift Shop,Gym,Gym / Fitness Center,Hostel,Hotel,Hotel Bar,IT Services,Ice Cream Shop,Indian Restaurant,Indonesian Restaurant,Intersection,Japanese Restaurant,Juice Bar,Korean Restaurant,Lake,Lounge,Market,Massage Studio,Movie Theater,Multiplex,Noodle House,Park,Photography Studio,Pizza Place,Pub,Restaurant,Sandwich Place,Seafood Restaurant,Shopping Mall,Shopping Plaza,Snack Place,Soup Place,Spa,Steakhouse,Supermarket,Sushi Restaurant,Szechuan Restaurant,Tea Room,Thai Restaurant,Vietnamese Restaurant,Warehouse Store
0,Ahlone,0.00,0.000000,0.00,0.000000,0.00,0.0000,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.0,0.0,0.200000,0.000000,0.0,0.000000,0.000000,0.00,0.000,0.000000,0.000000,0.000000,0.000000,0.000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.2,0.0,0.000000,0.000000,0.0000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000,0.000000,0.000000,0.000000,0.2,0.000000,0.0,0.000000,0.000000,0.0000,0.000000,0.000000,0.200000,0.000000,0.00,0.000000,0.0000,0.000000,0.00,0.000000,0.200000,0.000000,0.0,0.000000,0.00,0.00,0.000000,0.000000,0.0,0.000,0.000000,0.000,0.000000
1,Bahan,0.04,0.000000,0.04,0.040000,0.04,0.0000,0.0,0.040000,0.040000,0.040000,0.000000,0.0,0.00,0.000000,0.040000,0.000000,0.040000,0.000000,0.04,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.040000,0.00,0.000,0.000000,0.000000,0.000000,0.000000,0.000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.0000,0.040000,0.000000,0.000000,0.040000,0.000000,0.0,0.040000,0.000,0.000000,0.000000,0.080000,0.0,0.120000,0.0,0.000000,0.000000,0.0000,0.000000,0.000000,0.040000,0.000000,0.00,0.000000,0.0400,0.000000,0.04,0.000000,0.000000,0.000000,0.0,0.000000,0.04,0.04,0.000000,0.000000,0.0,0.000,0.040000,0.000,0.000000
2,Botahtaung,0.00,0.000000,0.00,0.000000,0.00,0.0625,0.0,0.000000,0.000000,0.062500,0.000000,0.0,0.00,0.000000,0.000000,0.000000,0.000000,0.187500,0.00,0.0,0.0,0.062500,0.062500,0.0,0.000000,0.000000,0.00,0.000,0.062500,0.000000,0.062500,0.000000,0.000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.0625,0.000000,0.000000,0.000000,0.062500,0.000000,0.0,0.000000,0.000,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.0625,0.000000,0.000000,0.000000,0.000000,0.00,0.062500,0.0625,0.000000,0.00,0.000000,0.062500,0.000000,0.0,0.000000,0.00,0.00,0.000000,0.000000,0.0,0.000,0.062500,0.000,0.000000
3,Dagon,0.00,0.000000,0.00,0.200000,0.00,0.0000,0.0,0.066667,0.000000,0.000000,0.000000,0.0,0.00,0.000000,0.066667,0.000000,0.000000,0.000000,0.00,0.0,0.0,0.000000,0.133333,0.0,0.000000,0.000000,0.00,0.000,0.000000,0.000000,0.000000,0.000000,0.000,0.000000,0.000000,0.000000,0.066667,0.0,0.000000,0.0,0.0,0.000000,0.066667,0.0000,0.000000,0.000000,0.000000,0.066667,0.000000,0.0,0.000000,0.000,0.066667,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.0000,0.000000,0.000000,0.066667,0.066667,0.00,0.000000,0.0000,0.133333,0.00,0.000000,0.000000,0.000000,0.0,0.000000,0.00,0.00,0.000000,0.000000,0.0,0.000,0.000000,0.000,0.000000
4,Dagon Myothit (South),0.00,0.000000,0.00,0.000000,0.00,0.0000,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.00,0.000,0.000000,0.000000,0.000000,0.000000,0.000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.000

<h1> 3. Methodology</h1>
<p> It is necessary to observed the most common type of business in Yangon and their distribution in different location, so, I used the sorted function to generate the result. The following figure show the top 5 common type of business in different districts within Yangon Division.</p>
<p> Due to the numerous categories for specific business types, I clustered into 15 by using K-mean clustering algorithm to identify the specific location and distribution. <p/>

In [196]:
num_top_venues = 5

for hood in yangon_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = yangon_grouped[yangon_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')
    



----Ahlone----
           venue  freq
0           Café   0.2
1     Food Stand   0.2
2      Juice Bar   0.2
3  Shopping Mall   0.2
4   Noodle House   0.2


----Bahan----
                 venue  freq
0    Korean Restaurant  0.12
1  Japanese Restaurant  0.08
2           Whisky Bar  0.04
3      Bubble Tea Shop  0.04
4       Ice Cream Shop  0.04


----Botahtaung----
                venue  freq
0  Burmese Restaurant  0.19
1         Pizza Place  0.06
2       Shopping Mall  0.06
3           Gift Shop  0.06
4      Massage Studio  0.06


----Dagon----
                venue  freq
0    Asian Restaurant  0.20
1  Chinese Restaurant  0.13
2          Restaurant  0.13
3     Bubble Tea Shop  0.07
4               Hotel  0.07


----Dagon Myothit (South)----
               venue  freq
0  Indian Restaurant   1.0
1         Whisky Bar   0.0
2       Intersection   0.0
3      Movie Theater   0.0
4     Massage Studio   0.0


----Dawbon----
                 venue  freq
0          Bus Station   1.0
1           Whi

In [197]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]
print('So far okay')

So far okay


In [198]:
num_top_venues = 5

indicators = ['st', 'nd', 'rd']

columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = yangon_grouped['Neighborhood']

for ind in np.arange(yangon_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(yangon_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head(30)


,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
0,Ahlone,Café,Shopping Mall,Food Stand,Noodle House,Juice Bar
1,Bahan,Korean Restaurant,Japanese Restaurant,Whisky Bar,Bar,Gym
2,Botahtaung,Burmese Restaurant,Dim Sum Restaurant,Chinese Restaurant,Massage Studio,Pizza Place
3,Dagon,Asian Restaurant,Chinese Restaurant,Restaurant,Bubble Tea Shop,Fast Food Restaurant
4,Dagon Myothit (South),Indian Restaurant,Dim Sum Restaurant,Chinese Restaurant,Clothing Store,Cocktail Bar
5,Dawbon,Bus Station,Dim Sum Restaurant,Chinese Restaurant,Clothing Store,Cocktail Bar
6,Hlaing,BBQ Joint,Park,Fast Food Restaurant,Warehouse Store,Dessert Shop
7,Insein,Department Store,Warehouse Store,Dim Sum Restaurant,Chinese Restaurant,Clothing Store
8,Kamayut,Shopping Mall,Snack Place,Korean Restaurant,Clothing Store,Noodle House
9,Kyauktada,Bar,Hotel,Burmese Restaurant,Restaurant,Bed & Breakfast


In [199]:
kclusters = 15

yangon_grouped_clustering = yangon_grouped.drop('Neighborhood', 1)

kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(yangon_grouped_clustering)

kmeans.labels_[0:10]


array([13,  1,  1,  1,  3,  2, 12,  5,  1,  1], dtype=int32)

In [200]:
yangon_merged = yangon_grouped

yangon_merged['Cluster Label'] = kmeans.labels_

yangon_merged = yangon_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on ='Neighborhood')

yangon_merged.head()


,Neighborhood,Whisky Bar,Accessories Store,American Restaurant,Asian Restaurant,Athletics & Sports,Australian Restaurant,Auto Workshop,BBQ Joint,Bakery,Bar,Bed & Breakfast,Bistro,Boxing Gym,Breakfast Spot,Bubble Tea Shop,Buffet,Burger Joint,Burmese Restaurant,Burrito Place,Bus Station,Bus Stop,Café,Chinese Restaurant,Clothing Store,Cocktail Bar,Coffee Shop,Convenience Store,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Donut Shop,Dumpling Restaurant,Electronics Store,Fast Food Restaurant,Fish Market,Food Court,Food Stand,Food Truck,Fried Chicken Joint,Garden,Gift Shop,Gym,Gym / Fitness Center,Hostel,Hotel,Hotel Bar,IT Services,Ice Cream Shop,Indian Restaurant,Indonesian Restaurant,Intersection,Japanese Restaurant,Juice Bar,Korean Restaurant,Lake,Lounge,Market,Massage Studio,Movie Theater,Multiplex,Noodle House,Park,Photography Studio,Pizza Place,Pub,Restaurant,Sandwich Place,Seafood Restaurant,Shopping Mall,Shopping Plaza,Snack Place,Soup Place,Spa,Steakhouse,Supermarket,Sushi Restaurant,Szechuan Restaurant,Tea Room,Thai Restaurant,Vietnamese Restaurant,Warehouse Store,Cluster Label,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
0,Ahlone,0.00,0.0,0.00,0.00,0.00,0.0000,0.0,0.000000,0.00,0.0000,0.0,0.0,0.0,0.0,0.000000,0.0,0.00,0.0000,0.00,0.0,0.0,0.2000,0.000000,0.0,0.0,0.00,0.0,0.0,0.0000,0.0,0.0000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.2,0.0,0.0,0.000000,0.0000,0.00,0.0,0.0,0.000000,0.0,0.0,0.00,0.0,0.000000,0.0,0.00,0.2,0.00,0.0,0.0,0.0,0.0000,0.0,0.0,0.200000,0.000000,0.0,0.0000,0.0000,0.000000,0.00,0.0,0.2000,0.0,0.0,0.0,0.00,0.00,0.0,0.0,0.0,0.0,0.0000,0.0,0.0,13,Café,Shopping Mall,Food Stand,Noodle House,Juice Bar
1,Bahan,0.04,0.0,0.04,0.04,0.04,0.0000,0.0,0.040000,0.04,0.0400,0.0,0.0,0.0,0.0,0.040000,0.0,0.04,0.0000,0.04,0.0,0.0,0.0000,0.000000,0.0,0.0,0.04,0.0,0.0,0.0000,0.0,0.0000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0000,0.04,0.0,0.0,0.040000,0.0,0.0,0.04,0.0,0.000000,0.0,0.08,0.0,0.12,0.0,0.0,0.0,0.0000,0.0,0.0,0.040000,0.000000,0.0,0.0000,0.0400,0.000000,0.04,0.0,0.0000,0.0,0.0,0.0,0.04,0.04,0.0,0.0,0.0,0.0,0.0400,0.0,0.0,1,Korean Restaurant,Japanese Restaurant,Whisky Bar,Bar,Gym
2,Botahtaung,0.00,0.0,0.00,0.00,0.00,0.0625,0.0,0.000000,0.00,0.0625,0.0,0.0,0.0,0.0,0.000000,0.0,0.00,0.1875,0.00,0.0,0.0,0.0625,0.062500,0.0,0.0,0.00,0.0,0.0,0.0625,0.0,0.0625,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0625,0.00,0.0,0.0,0.062500,0.0,0.0,0.00,0.0,0.000000,0.0,0.00,0.0,0.00,0.0,0.0,0.0,0.0625,0.0,0.0,0.000000,0.000000,0.0,0.0625,0.0625,0.000000,0.00,0.0,0.0625,0.0,0.0,0.0,0.00,0.00,0.0,0.0,0.0,0.0,0.0625,0.0,0.0,1,Burmese Restaurant,Dim Sum Restaurant,Chinese Restaurant,Massage Studio,Pizza Place
3,Dagon,0.00,0.0,0.00,0.20,0.00,0.0000,0.0,0.066667,0.00,0.0000,0.0,0.0,0.0,0.0,0.066667,0.0,0.00,0.0000,0.00,0.0,0.0,0.0000,0.133333,0.0,0.0,0.00,0.0,0.0,0.0000,0.0,0.0000,0.0,0.0,0.0,0.0,0.0,0.066667,0.0,0.0,0.0,0.0,0.0,0.066667,0.0000,0.00,0.0,0.0,0.066667,0.0,0.0,0.00,0.0,0.066667,0.0,0.00,0.0,0.00,0.0,0.0,0.0,0.0000,0.0,0.0,0.066667,0.066667,0.0,0.0000,0.0000,0.133333,0.00,0.0,0.0000,0.0,0.0,0.0,0.00,0.00,0.0,0.0,0.0,0.0,0.0000,0.0,0.0,1,Asian Restaurant,Chinese Restaurant,Restaurant,Bubble Tea Shop,Fast Food Restaurant
4,Dagon Myothit (South),0.00,0.0,0.00,0.00,0.00,0.0000,0.0,0.000000,0.00,0.0000,0.0,0.0,0.0,0.0,0.000000,0.0,0.00,0.0000,0.00,0.0,0.0,0.0000,0.000000,0.0,0.0,0.00,0.0,0.0,0.0000,0.0,0.0000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0000,0.00,0.0,0.0,0.000000,0.0,0.0,0.00,1.0,0.000000,0.0,0.00,0.0,0.00,0.0,0.0,0.0,0.0000,0.0,0.0,0.000000,0.000000,0.0,0.0000,0.0000,0.000000,0.00,0.0,0.0000,0.0,0.0,0.0,0.00,0.00,0.0,0.0,0.0,0.0,0.0000,0.0,0.0,3,Indian Restaurant,Dim Sum Restaurant,Chinese Restaurant,Clothing Store,Cocktail Bar


In [201]:
df.rename(columns={'District':'Neighborhood'}, inplace=True)
df2 = pd.DataFrame(df)
df2.head(10)

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Ahlone,16.782398,96.127863,Sein Gay Har @ Pyay Road,16.781087,96.127681,Shopping Mall
1,Ahlone,16.782398,96.127863,Urban 86 Food Court,16.782257,96.126869,Food Stand
2,Ahlone,16.782398,96.127863,Meringue Café,16.786160,96.128021,Café
3,Ahlone,16.782398,96.127863,Ahlone Drink,16.786057,96.129984,Juice Bar
4,Ahlone,16.782398,96.127863,ေရြွနန္းေတာ္ မုန့္တီ,16.784361,96.132085,Noodle House
5,Bahan,16.815427,96.156112,Manana,16.817201,96.156393,Burrito Place
6,Bahan,16.815427,96.156112,Real Fitness Gym,16.816024,96.153980,Athletics & Sports
7,Bahan,16.815427,96.156112,Yangon Bakehouse,16.816680,96.156155,Sandwich Place
8,Bahan,16.815427,96.156112,Golden Duck (Sayar San Road),16.816230,96.157350,Asian Restaurant
9,Bahan,16.815427,96.156112,SORABOL,16.817610,96.153924,Korean Restaurant


In [203]:
yangon_merged2 = pd.merge(df,yangon_merged, on = 'Neighborhood')
yangon_merged2.head(30)

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category,Whisky Bar,Accessories Store,American Restaurant,Asian Restaurant,Athletics & Sports,Australian Restaurant,Auto Workshop,BBQ Joint,Bakery,Bar,Bed & Breakfast,Bistro,Boxing Gym,Breakfast Spot,Bubble Tea Shop,Buffet,Burger Joint,Burmese Restaurant,Burrito Place,Bus Station,Bus Stop,Café,Chinese Restaurant,Clothing Store,Cocktail Bar,Coffee Shop,Convenience Store,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Donut Shop,Dumpling Restaurant,Electronics Store,Fast Food Restaurant,Fish Market,Food Court,Food Stand,Food Truck,Fried Chicken Joint,Garden,Gift Shop,Gym,Gym / Fitness Center,Hostel,Hotel,Hotel Bar,IT Services,Ice Cream Shop,Indian Restaurant,Indonesian Restaurant,Intersection,Japanese Restaurant,Juice Bar,Korean Restaurant,Lake,Lounge,Market,Massage Studio,Movie Theater,Multiplex,Noodle House,Park,Photography Studio,Pizza Place,Pub,Restaurant,Sandwich Place,Seafood Restaurant,Shopping Mall,Shopping Plaza,Snack Place,Soup Place,Spa,Steakhouse,Supermarket,Sushi Restaurant,Szechuan Restaurant,Tea Room,Thai Restaurant,Vietnamese Restaurant,Warehouse Store,Cluster Label,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
0,Ahlone,16.782398,96.127863,Sein Gay Har @ Pyay Road,16.781087,96.127681,Shopping Mall,0.00,0.0,0.00,0.00,0.00,0.0,0.0,0.00,0.00,0.00,0.0,0.0,0.0,0.0,0.00,0.0,0.00,0.0,0.00,0.0,0.0,0.2,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.2,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.00,0.0,0.0,0.00,0.0,0.0,0.0,0.00,0.2,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.20,0.0,0.0,0.0,0.00,0.0,0.00,0.0,0.2,0.0,0.0,0.0,0.00,0.00,0.0,0.0,0.0,0.0,0.00,0.0,0.0,13,Café,Shopping Mall,Food Stand,Noodle House,Juice Bar
1,Ahlone,16.782398,96.127863,Urban 86 Food Court,16.782257,96.126869,Food Stand,0.00,0.0,0.00,0.00,0.00,0.0,0.0,0.00,0.00,0.00,0.0,0.0,0.0,0.0,0.00,0.0,0.00,0.0,0.00,0.0,0.0,0.2,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.2,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.00,0.0,0.0,0.00,0.0,0.0,0.0,0.00,0.2,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.20,0.0,0.0,0.0,0.00,0.0,0.00,0.0,0.2,0.0,0.0,0.0,0.00,0.00,0.0,0.0,0.0,0.0,0.00,0.0,0.0,13,Café,Shopping Mall,Food Stand,Noodle House,Juice Bar
2,Ahlone,16.782398,96.127863,Meringue Café,16.786160,96.128021,Café,0.00,0.0,0.00,0.00,0.00,0.0,0.0,0.00,0.00,0.00,0.0,0.0,0.0,0.0,0.00,0.0,0.00,0.0,0.00,0.0,0.0,0.2,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.2,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.00,0.0,0.0,0.00,0.0,0.0,0.0,0.00,0.2,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.20,0.0,0.0,0.0,0.00,0.0,0.00,0.0,0.2,0.0,0.0,0.0,0.00,0.00,0.0,0.0,0.0,0.0,0.00,0.0,0.0,13,Café,Shopping Mall,Food Stand,Noodle House,Juice Bar
3,Ahlone,16.782398,96.127863,Ahlone Drink,16.786057,96.129984,Juice Bar,0.00,0.0,0.00,0.00,0.00,0.0,0.0,0.00,0.00,0.00,0.0,0.0,0.0,0.0,0.00,0.0,0.00,0.0,0.00,0.0,0.0,0.2,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.2,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.00,0.0,0.0,0.00,0.0,0.0,0.0,0.00,0.2,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.20,0.0,0.0,0.0,0.00,0.0,0.00,0.0,0.2,0.0,0.0,0.0,0.00,0.00,0.0,0.0,0.0,0.0,0.00,0.0,0.0,13,Café,Shopping Mall,Food Stand,Noodle House,Juice Bar
4,Ahlone,16.782398,96.127863,ေရြွနန္းေတာ္ မုန့္တီ,16.784361,96.132085,Noodle House,0.00,0.0,0.00,0.00,0.00,0.0,0.0,0.00,0.00,0.00,0.0,0.0,0.0,0.0,0.00,0.0,0.00,0.0,0.00,0.0,0.0,0.2,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.2,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.00,0.0,0.0,0.00,0.0,0.0,0.0,0.00,0.2,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.20,0.0,0.0,0.0,0.00,0.0,0.00,0.0,0.2,0.0,0.0,0.0,0.00,0.00,0.0,0.0,0.0,0.0,0.00,0.0,0.0,13,Café,Shopping Mall,Food Stand,Noodle House,Juice Bar
5,Bahan,16.815427,96.156112,Manana,16.817201,96.156393,Burrito Place,0.04,0.0,0.04,0.04,0.04,0.0,0.0,0.04,0.04,0.04,0.0,0.0,0.0,0.0,0.04,0.0,0.04,0.0,0.04,0.0,0.0,0.0,0.0,0.0,0.0,0.04,0.0,0.0,0.

In [204]:
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

markers_colors = []
for lat, lon, poi, cluster in zip(yangon_merged2['Neighborhood Latitude'], 
                                  yangon_merged2['Neighborhood Longitude'], 
                                  yangon_merged2['Neighborhood'], 
                                  yangon_merged2['Cluster Label']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

<h1> 4. Result</h1>

In [255]:
yangon_merged2.loc[yangon_merged2['Cluster Label'] == 0, yangon_merged2.columns[[0,1,2,4] + list(range(6, yangon_merged2.shape[1]))]]

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue Latitude,Venue Category,Whisky Bar,Accessories Store,American Restaurant,Asian Restaurant,Athletics & Sports,Australian Restaurant,Auto Workshop,BBQ Joint,Bakery,Bar,Bed & Breakfast,Bistro,Boxing Gym,Breakfast Spot,Bubble Tea Shop,Buffet,Burger Joint,Burmese Restaurant,Burrito Place,Bus Station,Bus Stop,Café,Chinese Restaurant,Clothing Store,Cocktail Bar,Coffee Shop,Convenience Store,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Donut Shop,Dumpling Restaurant,Electronics Store,Fast Food Restaurant,Fish Market,Food Court,Food Stand,Food Truck,Fried Chicken Joint,Garden,Gift Shop,Gym,Gym / Fitness Center,Hostel,Hotel,Hotel Bar,IT Services,Ice Cream Shop,Indian Restaurant,Indonesian Restaurant,Intersection,Japanese Restaurant,Juice Bar,Korean Restaurant,Lake,Lounge,Market,Massage Studio,Movie Theater,Multiplex,Noodle House,Park,Photography Studio,Pizza Place,Pub,Restaurant,Sandwich Place,Seafood Restaurant,Shopping Mall,Shopping Plaza,Snack Place,Soup Place,Spa,Steakhouse,Supermarket,Sushi Restaurant,Szechuan Restaurant,Tea Room,Thai Restaurant,Vietnamese Restaurant,Warehouse Store,Cluster Label,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
170,Mingalar Taung Nyunt,16.788908,96.167859,16.789050,Restaurant,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.25,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.25,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.25,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.25,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,Hotel,Restaurant,Department Store,Boxing Gym,Warehouse Store
171,Mingalar Taung Nyunt,16.788908,96.167859,16.788478,Department Store,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.25,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.25,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.25,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.25,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,Hotel,Restaurant,Department Store,Boxing Gym,Warehouse Store
172,Mingalar Taung Nyunt,16.788908,96.167859,16.792134,Boxing Gym,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.25,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.25,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.25,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.25,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,Hotel,Restaurant,Department Store,Boxing Gym,Warehouse Store
173,Mingalar Taung Nyunt,16.788908,96.167859,16.792520,Hotel,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.25,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.25,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.25,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.25,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,Hotel,Restaurant,Department Store,Boxing Gym,Warehouse Store


In [206]:
yangon_merged2.loc[yangon_merged2['Cluster Label'] == 1, yangon_merged2.columns[[0,1,2,4] + list(range(6, yangon_merged2.shape[1]))]]

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue Latitude,Venue Category,Whisky Bar,Accessories Store,American Restaurant,Asian Restaurant,Athletics & Sports,Australian Restaurant,Auto Workshop,BBQ Joint,Bakery,Bar,Bed & Breakfast,Bistro,Boxing Gym,Breakfast Spot,Bubble Tea Shop,Buffet,Burger Joint,Burmese Restaurant,Burrito Place,Bus Station,Bus Stop,Café,Chinese Restaurant,Clothing Store,Cocktail Bar,Coffee Shop,Convenience Store,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Donut Shop,Dumpling Restaurant,Electronics Store,Fast Food Restaurant,Fish Market,Food Court,Food Stand,Food Truck,Fried Chicken Joint,Garden,Gift Shop,Gym,Gym / Fitness Center,Hostel,Hotel,Hotel Bar,IT Services,Ice Cream Shop,Indian Restaurant,Indonesian Restaurant,Intersection,Japanese Restaurant,Juice Bar,Korean Restaurant,Lake,Lounge,Market,Massage Studio,Movie Theater,Multiplex,Noodle House,Park,Photography Studio,Pizza Place,Pub,Restaurant,Sandwich Place,Seafood Restaurant,Shopping Mall,Shopping Plaza,Snack Place,Soup Place,Spa,Steakhouse,Supermarket,Sushi Restaurant,Szechuan Restaurant,Tea Room,Thai Restaurant,Vietnamese Restaurant,Warehouse Store,Cluster Label,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
5,Bahan,16.815427,96.156112,16.817201,Burrito Place,0.04,0.000000,0.04,0.040,0.04,0.0000,0.0,0.040000,0.040000,0.040000,0.000000,0.0,0.0,0.0,0.040000,0.000000,0.040000,0.000000,0.04,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.040000,0.0,0.000,0.000000,0.000000,0.000000,0.0,0.000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.0000,0.040000,0.000000,0.000000,0.040000,0.000000,0.0,0.040000,0.000,0.000000,0.000000,0.080000,0.0,0.120000,0.0,0.000000,0.000000,0.0000,0.000000,0.000000,0.040000,0.000000,0.0,0.000000,0.0400,0.000000,0.04,0.000000,0.000000,0.000000,0.0,0.000000,0.04,0.04,0.000000,0.000000,0.0,0.000,0.040000,0.000,0.000000,1,Korean Restaurant,Japanese Restaurant,Whisky Bar,Bar,Gym
6,Bahan,16.815427,96.156112,16.816024,Athletics & Sports,0.04,0.000000,0.04,0.040,0.04,0.0000,0.0,0.040000,0.040000,0.040000,0.000000,0.0,0.0,0.0,0.040000,0.000000,0.040000,0.000000,0.04,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.040000,0.0,0.000,0.000000,0.000000,0.000000,0.0,0.000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.0000,0.040000,0.000000,0.000000,0.040000,0.000000,0.0,0.040000,0.000,0.000000,0.000000,0.080000,0.0,0.120000,0.0,0.000000,0.000000,0.0000,0.000000,0.000000,0.040000,0.000000,0.0,0.000000,0.0400,0.000000,0.04,0.000000,0.000000,0.000000,0.0,0.000000,0.04,0.04,0.000000,0.000000,0.0,0.000,0.040000,0.000,0.000000,1,Korean Restaurant,Japanese Restaurant,Whisky Bar,Bar,Gym
7,Bahan,16.815427,96.156112,16.816680,Sandwich Place,0.04,0.000000,0.04,0.040,0.04,0.0000,0.0,0.040000,0.040000,0.040000,0.000000,0.0,0.0,0.0,0.040000,0.000000,0.040000,0.000000,0.04,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.040000,0.0,0.000,0.000000,0.000000,0.000000,0.0,0.000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.0000,0.040000,0.000000,0.000000,0.040000,0.000000,0.0,0.040000,0.000,0.000000,0.000000,0.080000,0.0,0.120000,0.0,0.000000,0.000000,0.0000,0.000000,0.000000,0.040000,0.000000,0.0,0.000000,0.0400,0.000000,0.04,0.000000,0.000000,0.000000,0.0,0.000000,0.04,0.04,0.000000,0.000000,0.0,0.000,0.040000,0.000,0.000000,1,Korean Restaurant,Japanese Restaurant,Whisky Bar,Bar,Gym
8,Bahan,16.815427,96.156112,16.816230,Asian Restaurant,0.04,0.000000,0.04,0.040,0.04,0.0000,0.0,0.040000,0.040000,0.040000,0.000000,0.0,0.0,0.0,0.040000,0.000000,0.040000,0.000000,0.04,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.040000,0.0,0.000,0.000000,0.000000,0.000000,0.0,0.000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.0000,0.040000,0.000000,0.000000,0.040000,0.000000,0.0,0.040000,0.000,0.000000,0.000000,0.080000,0.0,0.120000,0.0,0.000000,0.000000,0.0000,0.00000

In [207]:
yangon_merged2.loc[yangon_merged2['Cluster Label'] == 2, yangon_merged2.columns[[0,1,2,4] + list(range(6, yangon_merged2.shape[1]))]]

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue Latitude,Venue Category,Whisky Bar,Accessories Store,American Restaurant,Asian Restaurant,Athletics & Sports,Australian Restaurant,Auto Workshop,BBQ Joint,Bakery,Bar,Bed & Breakfast,Bistro,Boxing Gym,Breakfast Spot,Bubble Tea Shop,Buffet,Burger Joint,Burmese Restaurant,Burrito Place,Bus Station,Bus Stop,Café,Chinese Restaurant,Clothing Store,Cocktail Bar,Coffee Shop,Convenience Store,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Donut Shop,Dumpling Restaurant,Electronics Store,Fast Food Restaurant,Fish Market,Food Court,Food Stand,Food Truck,Fried Chicken Joint,Garden,Gift Shop,Gym,Gym / Fitness Center,Hostel,Hotel,Hotel Bar,IT Services,Ice Cream Shop,Indian Restaurant,Indonesian Restaurant,Intersection,Japanese Restaurant,Juice Bar,Korean Restaurant,Lake,Lounge,Market,Massage Studio,Movie Theater,Multiplex,Noodle House,Park,Photography Studio,Pizza Place,Pub,Restaurant,Sandwich Place,Seafood Restaurant,Shopping Mall,Shopping Plaza,Snack Place,Soup Place,Spa,Steakhouse,Supermarket,Sushi Restaurant,Szechuan Restaurant,Tea Room,Thai Restaurant,Vietnamese Restaurant,Warehouse Store,Cluster Label,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
62,Dawbon,16.781917,96.18435,16.780077,Bus Station,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2,Bus Station,Dim Sum Restaurant,Chinese Restaurant,Clothing Store,Cocktail Bar


In [208]:
yangon_merged2.loc[yangon_merged2['Cluster Label'] == 3, yangon_merged2.columns[[0,1,2,4] + list(range(6, yangon_merged2.shape[1]))]]

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue Latitude,Venue Category,Whisky Bar,Accessories Store,American Restaurant,Asian Restaurant,Athletics & Sports,Australian Restaurant,Auto Workshop,BBQ Joint,Bakery,Bar,Bed & Breakfast,Bistro,Boxing Gym,Breakfast Spot,Bubble Tea Shop,Buffet,Burger Joint,Burmese Restaurant,Burrito Place,Bus Station,Bus Stop,Café,Chinese Restaurant,Clothing Store,Cocktail Bar,Coffee Shop,Convenience Store,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Donut Shop,Dumpling Restaurant,Electronics Store,Fast Food Restaurant,Fish Market,Food Court,Food Stand,Food Truck,Fried Chicken Joint,Garden,Gift Shop,Gym,Gym / Fitness Center,Hostel,Hotel,Hotel Bar,IT Services,Ice Cream Shop,Indian Restaurant,Indonesian Restaurant,Intersection,Japanese Restaurant,Juice Bar,Korean Restaurant,Lake,Lounge,Market,Massage Studio,Movie Theater,Multiplex,Noodle House,Park,Photography Studio,Pizza Place,Pub,Restaurant,Sandwich Place,Seafood Restaurant,Shopping Mall,Shopping Plaza,Snack Place,Soup Place,Spa,Steakhouse,Supermarket,Sushi Restaurant,Szechuan Restaurant,Tea Room,Thai Restaurant,Vietnamese Restaurant,Warehouse Store,Cluster Label,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
61,Dagon Myothit (South),16.84016,96.22582,16.841605,Indian Restaurant,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3,Indian Restaurant,Dim Sum Restaurant,Chinese Restaurant,Clothing Store,Cocktail Bar


In [209]:
yangon_merged2.loc[yangon_merged2['Cluster Label'] == 4, yangon_merged2.columns[[0,1,2,4] + list(range(6, yangon_merged2.shape[1]))]]

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue Latitude,Venue Category,Whisky Bar,Accessories Store,American Restaurant,Asian Restaurant,Athletics & Sports,Australian Restaurant,Auto Workshop,BBQ Joint,Bakery,Bar,Bed & Breakfast,Bistro,Boxing Gym,Breakfast Spot,Bubble Tea Shop,Buffet,Burger Joint,Burmese Restaurant,Burrito Place,Bus Station,Bus Stop,Café,Chinese Restaurant,Clothing Store,Cocktail Bar,Coffee Shop,Convenience Store,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Donut Shop,Dumpling Restaurant,Electronics Store,Fast Food Restaurant,Fish Market,Food Court,Food Stand,Food Truck,Fried Chicken Joint,Garden,Gift Shop,Gym,Gym / Fitness Center,Hostel,Hotel,Hotel Bar,IT Services,Ice Cream Shop,Indian Restaurant,Indonesian Restaurant,Intersection,Japanese Restaurant,Juice Bar,Korean Restaurant,Lake,Lounge,Market,Massage Studio,Movie Theater,Multiplex,Noodle House,Park,Photography Studio,Pizza Place,Pub,Restaurant,Sandwich Place,Seafood Restaurant,Shopping Mall,Shopping Plaza,Snack Place,Soup Place,Spa,Steakhouse,Supermarket,Sushi Restaurant,Szechuan Restaurant,Tea Room,Thai Restaurant,Vietnamese Restaurant,Warehouse Store,Cluster Label,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
135,Kyimyindaing,16.814573,96.121879,16.812152,Fish Market,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4,Chinese Restaurant,Fish Market,Warehouse Store,Dim Sum Restaurant,Clothing Store
136,Kyimyindaing,16.814573,96.121879,16.810530,Chinese Restaurant,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4,Chinese Restaurant,Fish Market,Warehouse Store,Dim Sum Restaurant,Clothing Store


In [210]:
yangon_merged2.loc[yangon_merged2['Cluster Label'] == 5, yangon_merged2.columns[[0,1,2,4] + list(range(6, yangon_merged2.shape[1]))]]

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue Latitude,Venue Category,Whisky Bar,Accessories Store,American Restaurant,Asian Restaurant,Athletics & Sports,Australian Restaurant,Auto Workshop,BBQ Joint,Bakery,Bar,Bed & Breakfast,Bistro,Boxing Gym,Breakfast Spot,Bubble Tea Shop,Buffet,Burger Joint,Burmese Restaurant,Burrito Place,Bus Station,Bus Stop,Café,Chinese Restaurant,Clothing Store,Cocktail Bar,Coffee Shop,Convenience Store,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Donut Shop,Dumpling Restaurant,Electronics Store,Fast Food Restaurant,Fish Market,Food Court,Food Stand,Food Truck,Fried Chicken Joint,Garden,Gift Shop,Gym,Gym / Fitness Center,Hostel,Hotel,Hotel Bar,IT Services,Ice Cream Shop,Indian Restaurant,Indonesian Restaurant,Intersection,Japanese Restaurant,Juice Bar,Korean Restaurant,Lake,Lounge,Market,Massage Studio,Movie Theater,Multiplex,Noodle House,Park,Photography Studio,Pizza Place,Pub,Restaurant,Sandwich Place,Seafood Restaurant,Shopping Mall,Shopping Plaza,Snack Place,Soup Place,Spa,Steakhouse,Supermarket,Sushi Restaurant,Szechuan Restaurant,Tea Room,Thai Restaurant,Vietnamese Restaurant,Warehouse Store,Cluster Label,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
67,Insein,16.901771,96.095959,16.903452,Department Store,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5,Department Store,Warehouse Store,Dim Sum Restaurant,Chinese Restaurant,Clothing Store


In [211]:
yangon_merged2.loc[yangon_merged2['Cluster Label'] == 6, yangon_merged2.columns[[0,1,2,4] + list(range(6, yangon_merged2.shape[1]))]]

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue Latitude,Venue Category,Whisky Bar,Accessories Store,American Restaurant,Asian Restaurant,Athletics & Sports,Australian Restaurant,Auto Workshop,BBQ Joint,Bakery,Bar,Bed & Breakfast,Bistro,Boxing Gym,Breakfast Spot,Bubble Tea Shop,Buffet,Burger Joint,Burmese Restaurant,Burrito Place,Bus Station,Bus Stop,Café,Chinese Restaurant,Clothing Store,Cocktail Bar,Coffee Shop,Convenience Store,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Donut Shop,Dumpling Restaurant,Electronics Store,Fast Food Restaurant,Fish Market,Food Court,Food Stand,Food Truck,Fried Chicken Joint,Garden,Gift Shop,Gym,Gym / Fitness Center,Hostel,Hotel,Hotel Bar,IT Services,Ice Cream Shop,Indian Restaurant,Indonesian Restaurant,Intersection,Japanese Restaurant,Juice Bar,Korean Restaurant,Lake,Lounge,Market,Massage Studio,Movie Theater,Multiplex,Noodle House,Park,Photography Studio,Pizza Place,Pub,Restaurant,Sandwich Place,Seafood Restaurant,Shopping Mall,Shopping Plaza,Snack Place,Soup Place,Spa,Steakhouse,Supermarket,Sushi Restaurant,Szechuan Restaurant,Tea Room,Thai Restaurant,Vietnamese Restaurant,Warehouse Store,Cluster Label,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
176,Okkalapa (North),16.918775,96.163028,16.918842,Food Truck,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,6,Food Truck,Chinese Restaurant,Dim Sum Restaurant,Clothing Store,Cocktail Bar
177,Okkalapa (North),16.918775,96.163028,16.922843,Chinese Restaurant,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,6,Food Truck,Chinese Restaurant,Dim Sum Restaurant,Clothing Store,Cocktail Bar


In [212]:
yangon_merged2.loc[yangon_merged2['Cluster Label'] == 7, yangon_merged2.columns[[0,1,2,4] + list(range(6, yangon_merged2.shape[1]))]]

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue Latitude,Venue Category,Whisky Bar,Accessories Store,American Restaurant,Asian Restaurant,Athletics & Sports,Australian Restaurant,Auto Workshop,BBQ Joint,Bakery,Bar,Bed & Breakfast,Bistro,Boxing Gym,Breakfast Spot,Bubble Tea Shop,Buffet,Burger Joint,Burmese Restaurant,Burrito Place,Bus Station,Bus Stop,Café,Chinese Restaurant,Clothing Store,Cocktail Bar,Coffee Shop,Convenience Store,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Donut Shop,Dumpling Restaurant,Electronics Store,Fast Food Restaurant,Fish Market,Food Court,Food Stand,Food Truck,Fried Chicken Joint,Garden,Gift Shop,Gym,Gym / Fitness Center,Hostel,Hotel,Hotel Bar,IT Services,Ice Cream Shop,Indian Restaurant,Indonesian Restaurant,Intersection,Japanese Restaurant,Juice Bar,Korean Restaurant,Lake,Lounge,Market,Massage Studio,Movie Theater,Multiplex,Noodle House,Park,Photography Studio,Pizza Place,Pub,Restaurant,Sandwich Place,Seafood Restaurant,Shopping Mall,Shopping Plaza,Snack Place,Soup Place,Spa,Steakhouse,Supermarket,Sushi Restaurant,Szechuan Restaurant,Tea Room,Thai Restaurant,Vietnamese Restaurant,Warehouse Store,Cluster Label,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
178,Okkalapa (South),16.846249,96.179859,16.846639,Breakfast Spot,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.250000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.25,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.25,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.25,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,7,Chinese Restaurant,Photography Studio,Japanese Restaurant,Breakfast Spot,Café
179,Okkalapa (South),16.846249,96.179859,16.845130,Japanese Restaurant,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.250000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.25,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.25,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.25,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,7,Chinese Restaurant,Photography Studio,Japanese Restaurant,Breakfast Spot,Café
180,Okkalapa (South),16.846249,96.179859,16.843579,Chinese Restaurant,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.250000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.25,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.25,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.25,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,7,Chinese Restaurant,Photography Studio,Japanese Restaurant,Breakfast Spot,Café
181,Okkalapa (South),16.846249,96.179859,16.842762,Photography Studio,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.250000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.25,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.25,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.25,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,7,Chinese Restaurant,Photography Studio,Japanese Restaurant,Breakfast Spot,Café
265,Yankin,16.836010,96.162395,16.835114,Café,0.0,0.0,0.0,0.142857,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.285714,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.142857,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.142857,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.142857,0.0,0.0,0.0,0.142857,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,7,Breakfast Spot,Diner,Seafood Restaurant,Asian Restaurant,Soup Place
266,Yankin,16.836010,96.162395,16.836767,Soup Place,0.0,0.0,0.0,0.142857,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.285714,0

In [213]:
yangon_merged2.loc[yangon_merged2['Cluster Label'] == 8, yangon_merged2.columns[[0,1,2,4] + list(range(6, yangon_merged2.shape[1]))]]

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue Latitude,Venue Category,Whisky Bar,Accessories Store,American Restaurant,Asian Restaurant,Athletics & Sports,Australian Restaurant,Auto Workshop,BBQ Joint,Bakery,Bar,Bed & Breakfast,Bistro,Boxing Gym,Breakfast Spot,Bubble Tea Shop,Buffet,Burger Joint,Burmese Restaurant,Burrito Place,Bus Station,Bus Stop,Café,Chinese Restaurant,Clothing Store,Cocktail Bar,Coffee Shop,Convenience Store,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Donut Shop,Dumpling Restaurant,Electronics Store,Fast Food Restaurant,Fish Market,Food Court,Food Stand,Food Truck,Fried Chicken Joint,Garden,Gift Shop,Gym,Gym / Fitness Center,Hostel,Hotel,Hotel Bar,IT Services,Ice Cream Shop,Indian Restaurant,Indonesian Restaurant,Intersection,Japanese Restaurant,Juice Bar,Korean Restaurant,Lake,Lounge,Market,Massage Studio,Movie Theater,Multiplex,Noodle House,Park,Photography Studio,Pizza Place,Pub,Restaurant,Sandwich Place,Seafood Restaurant,Shopping Mall,Shopping Plaza,Snack Place,Soup Place,Spa,Steakhouse,Supermarket,Sushi Restaurant,Szechuan Restaurant,Tea Room,Thai Restaurant,Vietnamese Restaurant,Warehouse Store,Cluster Label,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
236,Tanyin,16.76378,96.25184,16.763908,Restaurant,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,8,Restaurant,Warehouse Store,Dessert Shop,Chinese Restaurant,Clothing Store


In [214]:
yangon_merged2.loc[yangon_merged2['Cluster Label'] == 9, yangon_merged2.columns[[0,1,2,4] + list(range(6, yangon_merged2.shape[1]))]]

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue Latitude,Venue Category,Whisky Bar,Accessories Store,American Restaurant,Asian Restaurant,Athletics & Sports,Australian Restaurant,Auto Workshop,BBQ Joint,Bakery,Bar,Bed & Breakfast,Bistro,Boxing Gym,Breakfast Spot,Bubble Tea Shop,Buffet,Burger Joint,Burmese Restaurant,Burrito Place,Bus Station,Bus Stop,Café,Chinese Restaurant,Clothing Store,Cocktail Bar,Coffee Shop,Convenience Store,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Donut Shop,Dumpling Restaurant,Electronics Store,Fast Food Restaurant,Fish Market,Food Court,Food Stand,Food Truck,Fried Chicken Joint,Garden,Gift Shop,Gym,Gym / Fitness Center,Hostel,Hotel,Hotel Bar,IT Services,Ice Cream Shop,Indian Restaurant,Indonesian Restaurant,Intersection,Japanese Restaurant,Juice Bar,Korean Restaurant,Lake,Lounge,Market,Massage Studio,Movie Theater,Multiplex,Noodle House,Park,Photography Studio,Pizza Place,Pub,Restaurant,Sandwich Place,Seafood Restaurant,Shopping Mall,Shopping Plaza,Snack Place,Soup Place,Spa,Steakhouse,Supermarket,Sushi Restaurant,Szechuan Restaurant,Tea Room,Thai Restaurant,Vietnamese Restaurant,Warehouse Store,Cluster Label,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
237,Tharkayta,16.793009,96.202959,16.793569,Café,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,9,Café,Dim Sum Restaurant,Chinese Restaurant,Clothing Store,Cocktail Bar


In [215]:
yangon_merged2.loc[yangon_merged2['Cluster Label'] == 10, yangon_merged2.columns[[0,1,2,4] + list(range(6, yangon_merged2.shape[1]))]]

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue Latitude,Venue Category,Whisky Bar,Accessories Store,American Restaurant,Asian Restaurant,Athletics & Sports,Australian Restaurant,Auto Workshop,BBQ Joint,Bakery,Bar,Bed & Breakfast,Bistro,Boxing Gym,Breakfast Spot,Bubble Tea Shop,Buffet,Burger Joint,Burmese Restaurant,Burrito Place,Bus Station,Bus Stop,Café,Chinese Restaurant,Clothing Store,Cocktail Bar,Coffee Shop,Convenience Store,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Donut Shop,Dumpling Restaurant,Electronics Store,Fast Food Restaurant,Fish Market,Food Court,Food Stand,Food Truck,Fried Chicken Joint,Garden,Gift Shop,Gym,Gym / Fitness Center,Hostel,Hotel,Hotel Bar,IT Services,Ice Cream Shop,Indian Restaurant,Indonesian Restaurant,Intersection,Japanese Restaurant,Juice Bar,Korean Restaurant,Lake,Lounge,Market,Massage Studio,Movie Theater,Multiplex,Noodle House,Park,Photography Studio,Pizza Place,Pub,Restaurant,Sandwich Place,Seafood Restaurant,Shopping Mall,Shopping Plaza,Snack Place,Soup Place,Spa,Steakhouse,Supermarket,Sushi Restaurant,Szechuan Restaurant,Tea Room,Thai Restaurant,Vietnamese Restaurant,Warehouse Store,Cluster Label,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
174,Mingalardon,16.94914,96.12795,16.944849,Bus Stop,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,10,Bus Stop,Lake,Dessert Shop,Chinese Restaurant,Clothing Store
175,Mingalardon,16.94914,96.12795,16.944729,Lake,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,10,Bus Stop,Lake,Dessert Shop,Chinese Restaurant,Clothing Store


In [216]:
yangon_merged2.loc[yangon_merged2['Cluster Label'] == 11, yangon_merged2.columns[[0,1,2,4] + list(range(6, yangon_merged2.shape[1]))]]

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue Latitude,Venue Category,Whisky Bar,Accessories Store,American Restaurant,Asian Restaurant,Athletics & Sports,Australian Restaurant,Auto Workshop,BBQ Joint,Bakery,Bar,Bed & Breakfast,Bistro,Boxing Gym,Breakfast Spot,Bubble Tea Shop,Buffet,Burger Joint,Burmese Restaurant,Burrito Place,Bus Station,Bus Stop,Café,Chinese Restaurant,Clothing Store,Cocktail Bar,Coffee Shop,Convenience Store,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Donut Shop,Dumpling Restaurant,Electronics Store,Fast Food Restaurant,Fish Market,Food Court,Food Stand,Food Truck,Fried Chicken Joint,Garden,Gift Shop,Gym,Gym / Fitness Center,Hostel,Hotel,Hotel Bar,IT Services,Ice Cream Shop,Indian Restaurant,Indonesian Restaurant,Intersection,Japanese Restaurant,Juice Bar,Korean Restaurant,Lake,Lounge,Market,Massage Studio,Movie Theater,Multiplex,Noodle House,Park,Photography Studio,Pizza Place,Pub,Restaurant,Sandwich Place,Seafood Restaurant,Shopping Mall,Shopping Plaza,Snack Place,Soup Place,Spa,Steakhouse,Supermarket,Sushi Restaurant,Szechuan Restaurant,Tea Room,Thai Restaurant,Vietnamese Restaurant,Warehouse Store,Cluster Label,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
208,Pazundaung,16.779648,96.174392,16.775627,Hostel,0.0,0.0,0.0,0.2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.2,0.6,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,11,Hotel,Asian Restaurant,Hostel,Warehouse Store,Dim Sum Restaurant
209,Pazundaung,16.779648,96.174392,16.781328,Asian Restaurant,0.0,0.0,0.0,0.2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.2,0.6,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,11,Hotel,Asian Restaurant,Hostel,Warehouse Store,Dim Sum Restaurant
210,Pazundaung,16.779648,96.174392,16.777305,Hotel,0.0,0.0,0.0,0.2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.2,0.6,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,11,Hotel,Asian Restaurant,Hostel,Warehouse Store,Dim Sum Restaurant
211,Pazundaung,16.779648,96.174392,16.776364,Hotel,0.0,0.0,0.0,0.2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.2,0.6,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,11,Hotel,Asian Restaurant,Hostel,Warehouse Store,Dim Sum Restaurant
212,Pazundaung,16.779648,96.174392,16.775683,Hotel,0.0,0.0,0.0,0.2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.2,0.6,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,11,Hotel,Asian Restaurant,Hostel,Warehouse Store,Dim Sum Restaurant


In [217]:
yangon_merged2.loc[yangon_merged2['Cluster Label'] == 12, yangon_merged2.columns[[0,1,2,4] + list(range(6, yangon_merged2.shape[1]))]]

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue Latitude,Venue Category,Whisky Bar,Accessories Store,American Restaurant,Asian Restaurant,Athletics & Sports,Australian Restaurant,Auto Workshop,BBQ Joint,Bakery,Bar,Bed & Breakfast,Bistro,Boxing Gym,Breakfast Spot,Bubble Tea Shop,Buffet,Burger Joint,Burmese Restaurant,Burrito Place,Bus Station,Bus Stop,Café,Chinese Restaurant,Clothing Store,Cocktail Bar,Coffee Shop,Convenience Store,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Donut Shop,Dumpling Restaurant,Electronics Store,Fast Food Restaurant,Fish Market,Food Court,Food Stand,Food Truck,Fried Chicken Joint,Garden,Gift Shop,Gym,Gym / Fitness Center,Hostel,Hotel,Hotel Bar,IT Services,Ice Cream Shop,Indian Restaurant,Indonesian Restaurant,Intersection,Japanese Restaurant,Juice Bar,Korean Restaurant,Lake,Lounge,Market,Massage Studio,Movie Theater,Multiplex,Noodle House,Park,Photography Studio,Pizza Place,Pub,Restaurant,Sandwich Place,Seafood Restaurant,Shopping Mall,Shopping Plaza,Snack Place,Soup Place,Spa,Steakhouse,Supermarket,Sushi Restaurant,Szechuan Restaurant,Tea Room,Thai Restaurant,Vietnamese Restaurant,Warehouse Store,Cluster Label,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
63,Hlaing,16.847934,96.125227,16.847453,BBQ Joint,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.25,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.25,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,12,BBQ Joint,Park,Fast Food Restaurant,Warehouse Store,Dessert Shop
64,Hlaing,16.847934,96.125227,16.846510,BBQ Joint,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.25,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.25,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,12,BBQ Joint,Park,Fast Food Restaurant,Warehouse Store,Dessert Shop
65,Hlaing,16.847934,96.125227,16.849542,Fast Food Restaurant,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.25,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.25,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,12,BBQ Joint,Park,Fast Food Restaurant,Warehouse Store,Dessert Shop
66,Hlaing,16.847934,96.125227,16.849415,Park,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.25,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.25,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,12,BBQ Joint,Park,Fast Food Restaurant,Warehouse Store,Dessert Shop


In [218]:
yangon_merged2.loc[yangon_merged2['Cluster Label'] == 13, yangon_merged2.columns[[0,1,2,4] + list(range(6, yangon_merged2.shape[1]))]]

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue Latitude,Venue Category,Whisky Bar,Accessories Store,American Restaurant,Asian Restaurant,Athletics & Sports,Australian Restaurant,Auto Workshop,BBQ Joint,Bakery,Bar,Bed & Breakfast,Bistro,Boxing Gym,Breakfast Spot,Bubble Tea Shop,Buffet,Burger Joint,Burmese Restaurant,Burrito Place,Bus Station,Bus Stop,Café,Chinese Restaurant,Clothing Store,Cocktail Bar,Coffee Shop,Convenience Store,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Donut Shop,Dumpling Restaurant,Electronics Store,Fast Food Restaurant,Fish Market,Food Court,Food Stand,Food Truck,Fried Chicken Joint,Garden,Gift Shop,Gym,Gym / Fitness Center,Hostel,Hotel,Hotel Bar,IT Services,Ice Cream Shop,Indian Restaurant,Indonesian Restaurant,Intersection,Japanese Restaurant,Juice Bar,Korean Restaurant,Lake,Lounge,Market,Massage Studio,Movie Theater,Multiplex,Noodle House,Park,Photography Studio,Pizza Place,Pub,Restaurant,Sandwich Place,Seafood Restaurant,Shopping Mall,Shopping Plaza,Snack Place,Soup Place,Spa,Steakhouse,Supermarket,Sushi Restaurant,Szechuan Restaurant,Tea Room,Thai Restaurant,Vietnamese Restaurant,Warehouse Store,Cluster Label,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
0,Ahlone,16.782398,96.127863,16.781087,Shopping Mall,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,13,Café,Shopping Mall,Food Stand,Noodle House,Juice Bar
1,Ahlone,16.782398,96.127863,16.782257,Food Stand,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,13,Café,Shopping Mall,Food Stand,Noodle House,Juice Bar
2,Ahlone,16.782398,96.127863,16.786160,Café,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,13,Café,Shopping Mall,Food Stand,Noodle House,Juice Bar
3,Ahlone,16.782398,96.127863,16.786057,Juice Bar,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,13,Café,Shopping Mall,Food Stand,Noodle House,Juice Bar
4,Ahlone,16.782398,96.127863,16.784361,Noodle House,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,13,Café,Shopping Mall,Food Stand,Noodle House,Juice Bar


In [219]:
yangon_merged2.loc[yangon_merged2['Cluster Label'] == 14, yangon_merged2.columns[[0,1,2,4] + list(range(6, yangon_merged2.shape[1]))]]

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue Latitude,Venue Category,Whisky Bar,Accessories Store,American Restaurant,Asian Restaurant,Athletics & Sports,Australian Restaurant,Auto Workshop,BBQ Joint,Bakery,Bar,Bed & Breakfast,Bistro,Boxing Gym,Breakfast Spot,Bubble Tea Shop,Buffet,Burger Joint,Burmese Restaurant,Burrito Place,Bus Station,Bus Stop,Café,Chinese Restaurant,Clothing Store,Cocktail Bar,Coffee Shop,Convenience Store,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Donut Shop,Dumpling Restaurant,Electronics Store,Fast Food Restaurant,Fish Market,Food Court,Food Stand,Food Truck,Fried Chicken Joint,Garden,Gift Shop,Gym,Gym / Fitness Center,Hostel,Hotel,Hotel Bar,IT Services,Ice Cream Shop,Indian Restaurant,Indonesian Restaurant,Intersection,Japanese Restaurant,Juice Bar,Korean Restaurant,Lake,Lounge,Market,Massage Studio,Movie Theater,Multiplex,Noodle House,Park,Photography Studio,Pizza Place,Pub,Restaurant,Sandwich Place,Seafood Restaurant,Shopping Mall,Shopping Plaza,Snack Place,Soup Place,Spa,Steakhouse,Supermarket,Sushi Restaurant,Szechuan Restaurant,Tea Room,Thai Restaurant,Vietnamese Restaurant,Warehouse Store,Cluster Label,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
232,Tamwe,16.810182,96.17646,16.809353,Chinese Restaurant,0.0,0.0,0.0,0.25,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.25,0.0,0.0,0.0,0.25,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.25,0.0,0.0,0.0,14,Chinese Restaurant,Tea Room,Asian Restaurant,Convenience Store,Warehouse Store
233,Tamwe,16.810182,96.17646,16.807496,Convenience Store,0.0,0.0,0.0,0.25,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.25,0.0,0.0,0.0,0.25,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.25,0.0,0.0,0.0,14,Chinese Restaurant,Tea Room,Asian Restaurant,Convenience Store,Warehouse Store
234,Tamwe,16.810182,96.17646,16.805975,Tea Room,0.0,0.0,0.0,0.25,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.25,0.0,0.0,0.0,0.25,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.25,0.0,0.0,0.0,14,Chinese Restaurant,Tea Room,Asian Restaurant,Convenience Store,Warehouse Store
235,Tamwe,16.810182,96.17646,16.806480,Asian Restaurant,0.0,0.0,0.0,0.25,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.25,0.0,0.0,0.0,0.25,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.25,0.0,0.0,0.0,14,Chinese Restaurant,Tea Room,Asian Restaurant,Convenience Store,Warehouse Store


<h1> 5. Discussion </h1>

<p>This analysis mainly focusses on seeking the appropriate location and type of restaurant for starting new restaurant business. Available data are used for better evidence-based decision for business planning. Location, top common business and distribution data alone could not generate concrete information for the real situation. Other factors should be included in this analysis to be more precises decision such as population density, customer's preferences, etc.
According to this analysis result, we can identify which type of restaurants are less competition and which place should be considered for new start up regardless consideration on other factors.</p>


<h1> 6. Conclusion </h1>

<p>This analysis exercise is the main objective for getting certificate for data science course. However, we need to make more practices in the real world to be able to generate great result and need to use more variables for better precision. Be that as it may, this is the good assignment for this course and we had learned a lot from our team mates.</p>